In [280]:
import torch
from torch import nn
from d2l import torch as d2l
import torchvision

batch_size = 256
def load_data(batch_size, isTrain=True):
    dataset = torchvision.datasets.FashionMNIST(root='../data',
                                                train=isTrain,
                                                transform=torchvision.transforms.ToTensor())
    data_loader = torch.utils.data.DataLoader(dataset, batch_size, shuffle=isTrain)
    for imgs, labels in data_loader:
        yield imgs.reshape(-1, 784).type(torch.float32), labels
# train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
train_iter = load_data(batch_size, True)
test_iter = load_data(batch_size, False)

In [281]:
class Accumulator:
    def __init__(self, n):
        self.data = [0.] * n
        
    def __getitem__(self, i):
        return self.data[i]
    
    def add(self, *args):
        self.data = [a + b for (a, b) in zip(self.data, args)]

### 定义ReLu函数

In [282]:
def relu(X):
    a = torch.zeros_like(X)
    return torch.max(X, a)

In [288]:
W1 = nn.Parameter(torch.randn(784, 256, requires_grad=True) * 0.01)
b1 = nn.Parameter(torch.zeros(256, requires_grad=True))
W2 = nn.Parameter(torch.randn(256, 10, requires_grad=True) * 0.01)
b2 = nn.Parameter(torch.zeros(10, requires_grad=True))

In [289]:
def softmax(X):
    exp_X = torch.exp(X)
    return exp_X / exp_X.sum(1, keepdim=True)

In [290]:
def net(X):
    X = X.reshape(-1, 784)
    H = relu(X @ W1 + b1)
    return (H @ W2 + b2)
#     return X @ W1 + b1

In [291]:
loss = nn.CrossEntropyLoss()

In [292]:

params = [W1, b1, W2, b2]
updater = torch.optim.SGD(params, 0.1)
num_epochs = 10
# d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, updater)

for epoch in range(epochs):
    acc = Accumulator(2)
    lacc = Accumulator(1)
    train_iter = load_data(batch_size, isTrain=True)
    test_iter = load_data(batch_size, isTrain=False)
    for x, y in train_iter:
        updater.zero_grad()
        y_hat = net(x)
        l = loss(y_hat, y)
        lacc.add(l)
        l.backward()
        updater.step()
    print(lacc[0])
    for x, y in test_iter:
        with torch.no_grad():
            y_hat = net(x)
            correct_num = (torch.argmax(y_hat, dim=1) == y).type(y.dtype).sum()
            acc.add(correct_num, len(y))
# #             print(softmax(y_hat))
    print(acc[0], acc[1])

tensor(243.5515, grad_fn=<AddBackward0>)
tensor(7645.) 10000.0
tensor(141.4802, grad_fn=<AddBackward0>)
tensor(7612.) 10000.0
tensor(121.3977, grad_fn=<AddBackward0>)
tensor(7907.) 10000.0
tensor(112.3852, grad_fn=<AddBackward0>)
tensor(8140.) 10000.0
tensor(106.7940, grad_fn=<AddBackward0>)
tensor(8307.) 10000.0
tensor(102.0197, grad_fn=<AddBackward0>)
tensor(8330.) 10000.0
tensor(98.2086, grad_fn=<AddBackward0>)
tensor(8236.) 10000.0
tensor(94.8536, grad_fn=<AddBackward0>)
tensor(8275.) 10000.0
tensor(92.7512, grad_fn=<AddBackward0>)
tensor(8379.) 10000.0
tensor(89.8452, grad_fn=<AddBackward0>)
tensor(8497.) 10000.0
